In [ ]:
import glob
import os
import iris
from iris.time import PartialDateTime
from iris.experimental.equalise_cubes import equalise_attributes

DATADIR = '/project/ciid/projects/PRECIS/worksheets/data_v2/EAS-22/'
# write outputs to own directory so that can be checked before upload under ciid
OUTDIR = f'{os.getenv("SCRATCH")}/cordex_inputs/'

# Make data for worksheet 4

In [ ]:
# Prep future data (2021-2050) from ESGF CORDEX files
# Note this assumes that only want to process the REMO2015 data.

shortnames = {'MOHC-HadGEM2-ES':'hadgem2-es', 'MPI-M-MPI-ESM-LR':'mpi-esm-lr'}

# Make future 2021-2050 data
future_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=2041) 
                            <= cell.point <= PartialDateTime(year=2060))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]
    for var in ['pr', 'tas']:

        # find all the files to consolidate 
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_rcp85_*_GERICS-REMO2015_*_mon_*.nc')
        print (f'input files: {flist}')

        # merge the cubes
        datacubes = iris.load(flist, future_time_constraint)
        equalise_attributes(datacubes)
        cube = datacubes.concatenate_cube()
        
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(OUTDIR, 'future', shortgcm + '.mon.2041_2060.GERICS-REMO2015.tm.C.nc')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(OUTDIR, 'future', shortgcm + '.mon.2041_2060.GERICS-REMO2015.pr.mmday-1.nc')
            iris.save(cube, outpath)
        print(f'saved file: {outpath}')

In [ ]:
# Prep baseline data (1986-2005) from 
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]
    for var in ['pr', 'tas']:

        # find all the files to consolidate 
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_historical_*_GERICS-REMO2015_*_mon_*.nc')
        flist = glob.glob(DATADIR + f'{var}_*_{gcmid}_historical_*_GERICS-REMO2015_*_mon_*.nc')
        print (f'input files: {flist}')

        # merge the cubes
        datacubes = iris.load(flist, historical_time_constraint)
        equalise_attributes(datacubes)
        cube = datacubes.concatenate_cube()
        
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.mon.1986_2005.GERICS-REMO2015.tm.C.nc')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.mon.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')
            iris.save(cube, outpath)
        print(f'saved file: {outpath}')

# Make data for Worksheet 5

Note that these datasets should be saved with compression to reduce their file size.  If for some reason this needs to be done during a PRECIS course, it is quicker to save them without compression using `iris.save(cube, outfile)` but the filesize is significantly larger.

Use: `/project/ciid/projects/PRECIS/worksheets/data/pp/wks5dataprep.py`

An example batch script is provided by `py_makedata.sh`

Then `nccopy -k nc4 -d 4 -s <infile> <outfile>`

In [ ]:
# This runs better on SPICE due to the large memory requirements
# Prep future data (2021-2050) from pp files

# Make future 2021-2050 data
for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(DATADIR, 'pp', runid)

    # find all the files from which to remove the rim
    flist = glob.glob(ppdir + '/*pa[mnop]*.pp')
    
    datacubes = iris.load(flist)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    
    for cube in tempcubelist:
        if cube.name() == 'air_temperature' and cube.cell_methods[0].method == 'mean':
            outpath = os.path.join(DATADIR, 'future', runid + '.day.2021_2050.tm.rr.C.nc')
            cube.convert_units('celsius')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
#             iris.fileformats.netcdf.save(cube, outpath, least_significant_digit=3,
#                                  netcdf_format='NETCDF4', complevel=4, zlib=True)
            iris.save(cube, outpath)
        elif cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(DATADIR, 'future', runid + '.day.2021_2050.pr.rr.mmday-1.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
#             iris.fileformats.netcdf.save(cube, outpath, least_significant_digit=3,
#                                  netcdf_format='NETCDF4', complevel=4, zlib=True)
            iris.save(cube, outpath)
        else:
            pass

In [ ]:
# Prep baseline data (1961-1990) from pp files

# Make baseline data
for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(DATADIR, 'pp', runid)

    # find all the files from which to remove the rim
    flist = glob.glob(ppdir + '/*pa[ghij]*.pp')
    
    datacubes = iris.load(flist)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    
    for cube in tempcubelist:
        if cube.name() == 'air_temperature' and cube.cell_methods[0].method == 'mean':
            cube.convert_units('celsius')
            outpath = os.path.join(DATADIR, 'historical', runid + '.day.1961_1990.tm.rr.C.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
#             iris.fileformats.netcdf.save(cube, outpath, least_significant_digit=3,
#                                  netcdf_format='NETCDF4', complevel=4, zlib=True)
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(DATADIR, 'historical', runid + '.day.1961_1990.pr.rr.mmday-1.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
#             iris.fileformats.netcdf.save(cube, outpath, least_significant_digit=3,
#                                  netcdf_format='NETCDF4', complevel=4, zlib=True)
            iris.save(cube, outpath)